# GPU 체크

In [ ]:
# %load device_test.py
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ","  # use CPU only
# del os.environ["CUDA_VISIBLE_DEVICES"]

import tensorflow as tf

print("tf.test.is_built_with_cuda():", tf.test.is_built_with_cuda())
print("tf.test.is_gpu_available():", tf.test.is_gpu_available())

sess = tf.Session()
if (sess.list_devices): # for tensorflow 1.3+
      for d in sess.list_devices():
          print(d.name)
sess.close()

# Undocumented feature
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


tf.test.is_built_with_cuda(): True
tf.test.is_gpu_available(): True
/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11660502352862854169
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9215183422
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10352115114446213185
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
]


# hello program

In [7]:
# %load hello.py
import tensorflow as tf

sess = tf.Session()
hello = tf.constant('Hello, TensorFlow!') # node (operation)
print(sess.run(hello)) # data flow graph의 실행


b'Hello, TensorFlow!'


# 데이터 플로우 그래프
- 하나의 데이터 플로 그래프 표현
- 연산을 나타내는 노드(node)와 노드간에 교환되는 데이터인 엣지(edge)로 구성된다.
- 노드간에 교환되는 데이터는 스칼라, 벡터, 행렬 등 다차원 배열을 의미하는 텐서(tensor)이다.
![Alt text](graph_test.png)

In [5]:
# %load graph_test.py
import tensorflow as tf

a = tf.constant(1.0)
b = tf.constant(2.0)
c = a + b

print(type(a), ",", a)
print(type(b), ",", b)
print(type(c), ",", c)

print("get_operations():")
for op in tf.get_default_graph().get_operations():
    print("op.name: ", op.name, ", op.type: ", op.type, ", op.inputs: ", [x for x in op.inputs],
              ", op.outputs: ", [x for x in op.outputs], sep="")
g = tf.get_default_graph()
print(a.graph is g)

print("Const:", g.get_operation_by_name("Const"))
print("Const.output[0]:", g.get_operation_by_name("Const").outputs[0])
print("Const:0", g.get_tensor_by_name("Const:0"))
print("a:", a, type(a))
print("a.name: ", a.name, ", a.op.name: ", a.op.name, ", a.value_index: ", a.value_index,
          ", a.shape: ", a.shape, ", a.dtype: ", a.dtype, sep="")

print("Const_1:", g.get_operation_by_name("Const_1"))
print("Const_1.outputs[0]:", g.get_operation_by_name("Const_1").outputs[0])
print("Const_1:0", g.get_tensor_by_name("Const_1:0"))
print("b:", b, type(b))
print("b.name: ", b.name, ", b.op.name: ", b.op.name, ", b.value_index: ", b.value_index,
          ", b.shape: ", b.shape, ", b.dtype: ", b.dtype, sep="")

print("add:", g.get_operation_by_name("add"))
print("add.outputs[0]:", g.get_operation_by_name("add").outputs[0])
print("add.inputs[0]:", g.get_operation_by_name("add").inputs[0])
print("add.inputs[1]:", g.get_operation_by_name("add").inputs[1])
print("c:", c, type(c))
print("c.name: ", c.name, ", c.op.name: ", c.op.name, ", c.value_index: ", c.value_index,
          ", c.shape: ", c.shape, ", c.dtype: ", c.dtype, sep="")

sess = tf.Session()
v = sess.run(g.get_tensor_by_name("Const:0"))
print("%s: %r\n" % (type(v), v))
sess.close()


<class 'tensorflow.python.framework.ops.Tensor'> , Tensor("Const_1:0", shape=(), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'> , Tensor("Const_2:0", shape=(), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'> , Tensor("add:0", shape=(), dtype=float32)
get_operations():
op.name: Const, op.type: Const, op.inputs: [], op.outputs: [<tf.Tensor 'Const:0' shape=() dtype=string>]
op.name: Const_1, op.type: Const, op.inputs: [], op.outputs: [<tf.Tensor 'Const_1:0' shape=() dtype=float32>]
op.name: Const_2, op.type: Const, op.inputs: [], op.outputs: [<tf.Tensor 'Const_2:0' shape=() dtype=float32>]
op.name: add, op.type: Add, op.inputs: [<tf.Tensor 'Const_1:0' shape=() dtype=float32>, <tf.Tensor 'Const_2:0' shape=() dtype=float32>], op.outputs: [<tf.Tensor 'add:0' shape=() dtype=float32>]
True
Const: name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_STRING
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_STRING
      tensor_sh

# 세션 관리

In [9]:
# %load session_test.py
import tensorflow as tf

# 텐서플로 기본 그래프 생성
a = tf.constant(1)
b = tf.constant(2)
c = a + b
addOp = tf.get_default_graph().get_operation_by_name("add")

# 세션 생성 및 닫기
sess = tf.Session()
v = sess.run(c)
print("%s: %r\n" % (type(v), v))
v = sess.run(addOp)
print("%s: %r\n" % (type(v), v))
sess.close()

# 세션 생성 및 자동 닫기
with tf.Session() as sess:
    v = sess.run(c)
    print("%s: %r\n" % (type(v), v))

# 기본 세션 (as_default())
sess = tf.Session()
with sess.as_default():
    v = addOp.run()
    print("%s: %r\n" % (type(v), v))
    v = c.eval()
    print("%s: %r\n" % (type(v), v))

sess.close()

# 기본 세션 (tf.InteractiveSession)
sess = tf.InteractiveSession()
v = addOp.run()
print("%s: %r\n" % (type(v), v))
v = c.eval()
print("%s: %r\n" % (type(v), v))
sess.close()


<class 'numpy.int32'>: 3

<class 'NoneType'>: None

<class 'numpy.int32'>: 3

<class 'NoneType'>: None

<class 'numpy.int32'>: 3

<class 'NoneType'>: None

<class 'numpy.int32'>: 3



In [11]:
# %load session_inout_test.py
import tensorflow as tf

sess = tf.Session()

a = tf.constant(1)
b = tf.constant(2)
c = a + b
x = tf.constant([10, 20])
y = tf.constant([1.0, 2.0])

v = sess.run(a)
print("%s: %r\n" % (type(v), v))

v = sess.run(c)
print("%s: %r\n" % (type(v), v))

v = sess.run(x)
print("%s: %r\n" % (type(v), v))

v = sess.run([x, y])
print("%s: %r\n" % (type(v), v))

import collections
MyData = collections.namedtuple('MyData', ['x', 'y'])
v = sess.run({'k1': MyData(x, y), 'k2': [y, x]})
print("%s: %r\n" % (type(v), v))

v = sess.run(c, feed_dict={a: 3, b: 4})
print(v)


<class 'numpy.int32'>: 1

<class 'numpy.int32'>: 3

<class 'numpy.ndarray'>: array([10, 20])

<class 'list'>: [array([10, 20]), array([1., 2.], dtype=float32)]

<class 'dict'>: {'k1': MyData(x=array([10, 20]), y=array([1., 2.], dtype=float32)), 'k2': [array([1., 2.], dtype=float32), array([10, 20])]}

7


# 상수
상수 텐서와 그 상수 텐서의 값을 채우는 텐서 연산을 생성한다.

In [13]:
# %load constant_test.py
import tensorflow as tf
import numpy as np

sess = tf.Session()

# 값 상수 텐서

c1 = tf.constant(1.0)
print(type(c1), ",", c1, ",", c1.shape, ",", c1.dtype, ",", sess.run(c1))
c2 = tf.constant([1.0, 2.0, 3.0])
print(c2.shape, ",", c2.dtype, ",", sess.run(c2))
c3 = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
print(c3.shape, ",", c3.dtype, ",", sess.run(c3))
c4 = tf.constant(np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]))
print(c4.shape, ",", c4.dtype, ",", sess.run(c4))
c5 = tf.constant(1.0, shape=(2,3))
print(c5.shape, ",", c5.dtype, ",", sess.run(c5))
c6 = tf.constant([1.0, 2.0], shape=(2,3))
print(c6.shape, ",", c6.dtype, ",", sess.run(c6))
c7 = tf.constant([1, 2, 3])
print(c7.dtype, ",", sess.run(c7))

# 특정값 상수 텐서

print(sess.run(tf.zeros([3])))
print(sess.run(tf.ones([3])))
print(sess.run(tf.fill([3], 2.0)))
print(sess.run(tf.zeros_like((c2))))
print(sess.run(tf.ones_like((c2))))

# 시퀀스

c8 = tf.range(1, 5)
print(type(c8), ",", c8, ",", sess.run(c8))
c9 = tf.range(5)
print(type(c9), ",", c9, ",", sess.run(c9))
print(sess.run(tf.lin_space(1.0, 3.0, 3)))

sess.close()


<class 'tensorflow.python.framework.ops.Tensor'> , Tensor("Const_11:0", shape=(), dtype=float32) , () , <dtype: 'float32'> , 1.0
(3,) , <dtype: 'float32'> , [1. 2. 3.]
(2, 3) , <dtype: 'float32'> , [[1. 2. 3.]
 [4. 5. 6.]]
(2, 3) , <dtype: 'float64'> , [[1. 2. 3.]
 [4. 5. 6.]]
(2, 3) , <dtype: 'float32'> , [[1. 1. 1.]
 [1. 1. 1.]]
(2, 3) , <dtype: 'float32'> , [[1. 2. 2.]
 [2. 2. 2.]]
<dtype: 'int32'> , [1 2 3]
[0. 0. 0.]
[1. 1. 1.]
[2. 2. 2.]
[0. 0. 0.]
[1. 1. 1.]
<class 'tensorflow.python.framework.ops.Tensor'> , Tensor("range:0", shape=(4,), dtype=int32) , [1 2 3 4]
<class 'tensorflow.python.framework.ops.Tensor'> , Tensor("range_1:0", shape=(5,), dtype=int32) , [0 1 2 3 4]
[1. 2. 3.]


# 변수
여러 세션간에 공유되어 영속적으로 존재하는 변수 (tf.Variable 클래스)

In [20]:
# %load variable_test.py
import tensorflow as tf

v1 = tf.get_variable("v1", (2,))
v2 = tf.get_variable("v2", (2,), dtype=tf.int32)
v3 = tf.get_variable("v3", dtype=tf.int32, initializer=tf.constant([3, 4]))
assign = v1.assign(tf.constant([3.0, 4.0]))
print(type(v1), v1)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run([v1, v2, v3, assign]))


<class 'tensorflow.python.ops.variables.RefVariable'> <tf.Variable 'v1:0' shape=(2,) dtype=float32_ref>
[array([3., 4.], dtype=float32), array([0, 0]), array([3, 4]), array([3., 4.], dtype=float32)]


# placeholder
노드에 전달되는 매개 변수 역할 (텐서의 일종)

In [2]:
# %load placeholder_test.py
import tensorflow as tf

a = tf.placeholder(tf.int16)
b = tf.placeholder(tf.int16)

print(type(a), a)

add = tf.add(a, b) # same with 'a + b'
mul = tf.multiply(a, b) # same with 'a * b'
print(mul)

with tf.Session() as sess:
    print("Addition with variables: %d" % sess.run(add, feed_dict={a:2, b:3}))
    print("Multiplication with variables: %d" % sess.run(mul, feed_dict={a:2, b:3}))


<class 'tensorflow.python.framework.ops.Tensor'> Tensor("Placeholder_2:0", dtype=int16)
Tensor("Mul_1:0", dtype=int16)
Addition with variables: 5
Multiplication with variables: 6


In [5]:
# %load matmul_test.py
import tensorflow as tf

a = tf.constant([[3., 3.]])  # 1 x 2 matrix
b = tf.constant([[2.], [2.]]) # 2 x 1 matrix

product = tf.matmul(a, b)

with tf.Session() as sess:
    print(sess.run(product))


[[12.]]


In [ ]:
# %load matmul_speed_test.py
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ","  # use CPU only
# del os.environ["CUDA_VISIBLE_DEVICES"]

import tensorflow as tf
import time

for size in [1000, 2000, 4000, 8000, 16000, 24000]:  # 12 MB, 48 MB, 192 MB, 768 MB, 3 GB, 6.75 GB
    a = tf.random_uniform((size, size), 0.0, 1.0)
    b = tf.random_uniform((size, size), 0.0, 1.0)
    c = tf.matmul(a, b)

    with tf.Session() as sess:
        start = time.time()
        sess.run(c)
        end = time.time()

    print("%s: %f 초" % (c.shape, end - start))
    a = b = c = None    # 쓰레기 수집
    tf.reset_default_graph()



# 최적화 문제 - 분석적 해법 (최적 해)

- 손실 함수    
    - x^4 - 3 x^3 + 2
- 풀이
    - f'(x) = 4 x^3 - 9 x^2 = 0
    - f(0) = 2
    - f(2.25) = 2.25^4 - 3*2.25^3 + 2 = -6.54296875

#  절차적 해법 (근사 해)

In [24]:
# %load gradient_descent_test.py
## 학습 알고리듬 설정
x = 6 # The algorithm starts at x=6
delta = 0.01 # step size
n = 50 # number of learning iterations

## 손실 함수
def f(x):
    return x ** 4 - 3 * x ** 3 + 2

## 수작업 미분 함수
def f_derivative(x):
    return 4 * x**3 - 9 * x**2

## 경사하강 학습
print("%3d: f(%f) = %f" % (0, x, f(x)))

for count in range(n):
    x -= delta * f_derivative(x)
    print("%3d: f(%f) = %f" % ((count+1), x, f(x)))


  0: f(6.000000) = 650.000000
  1: f(0.600000) = 1.481600
  2: f(0.623760) = 1.423309
  3: f(0.649069) = 1.357145
  4: f(0.676048) = 1.281943
  5: f(0.704822) = 1.196373
  6: f(0.735526) = 1.098924
  7: f(0.768299) = 0.987891
  8: f(0.803284) = 0.861373
  9: f(0.840625) = 0.717271
 10: f(0.880462) = 0.553317
 11: f(0.922930) = 0.367110
 12: f(0.968146) = 0.156197
 13: f(1.016205) = -0.081809
 14: f(1.067169) = -0.349059
 15: f(1.121052) = -0.647233
 16: f(1.177805) = -0.977246
 17: f(1.237300) = -1.338898
 18: f(1.299314) = -1.730494
 19: f(1.363512) = -2.148489
 20: f(1.429437) = -2.587232
 21: f(1.496503) = -3.038898
 22: f(1.564002) = -3.493712
 23: f(1.631123) = -3.940522
 24: f(1.696986) = -4.367716
 25: f(1.760688) = -4.764375
 26: f(1.821363) = -5.121467
 27: f(1.878240) = -5.432815
 28: f(1.930700) = -5.695626
 29: f(1.978309) = -5.910464
 30: f(2.020842) = -6.080715
 31: f(2.058275) = -6.211724
 32: f(2.090765) = -6.309823
 33: f(2.118607) = -6.381483
 34: f(2.142198) = -6.432

# 절차적 해법 (텐서플로)

In [28]:
# %load gradient_descent_tf_test.py
import tensorflow as tf

precision = 0.00001

## 최적화 함수 정의

x = tf.get_variable("x", initializer=6.0)
y = x**4 - 3 * x**3 + 2

## 경사 하강 알고리듬 설정

a = tf.get_variable("a", initializer=0.01) # learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(y)

# 변수 초기화
init = tf.global_variables_initializer()

## 훈련
sess = tf.Session()
sess.run(init)

count = 0
x_old = 0
x_new = sess.run(x)
print("%3d: f(%f) = %f, precision: %f" % (count, x_new, sess.run(y), x_new - x_old))

while abs(x_new - x_old) > precision:
    count += 1
    x_old = x_new
    sess.run(train)
    x_new = sess.run(x)
    print("%3d: f(%f) = %f, precision: %f" % (count, x_new, sess.run(y), x_new - x_old))

print("Local minimum occurs at", x_new)


  0: f(6.000000) = 650.000000, precision: 6.000000
  1: f(0.600000) = 1.481600, precision: -5.400000
  2: f(0.623760) = 1.423309, precision: 0.023760
  3: f(0.649069) = 1.357145, precision: 0.025309
  4: f(0.676048) = 1.281942, precision: 0.026978
  5: f(0.704822) = 1.196372, precision: 0.028774
  6: f(0.735526) = 1.098923, precision: 0.030704
  7: f(0.768299) = 0.987890, precision: 0.032773
  8: f(0.803284) = 0.861372, precision: 0.034985
  9: f(0.840625) = 0.717271, precision: 0.037341
 10: f(0.880462) = 0.553316, precision: 0.039837
 11: f(0.922930) = 0.367109, precision: 0.042467
 12: f(0.968146) = 0.156196, precision: 0.045216
 13: f(1.016205) = -0.081810, precision: 0.048060
 14: f(1.067170) = -0.349060, precision: 0.050964
 15: f(1.121052) = -0.647235, precision: 0.053883
 16: f(1.177805) = -0.977248, precision: 0.056753
 17: f(1.237300) = -1.338900, precision: 0.059495
 18: f(1.299314) = -1.730495, precision: 0.062014
 19: f(1.363513) = -2.148491, precision: 0.064199
 20: f(1.4

# 경사하강 학습법 - 단순 선형회귀

In [34]:
# %load simple_linear_regression_gd_tf_test.py
import tensorflow as tf

## 데이터 수집

x_data = [1, 2, 3]
y_data = [1, 2, 3]

## 예측 모델 정의

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
y = W * x_data + b

## 비용 함수, 최적화 함수 정의

cost = tf.reduce_mean(tf.square(y - y_data))
a = tf.Variable(0.1) # learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

# 변수 초기화
init = tf.global_variables_initializer()

## 훈련
sess = tf.Session()
sess.run(init)

for step in range(2001):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(cost), sess.run(W), sess.run(b))


0 0.07709669 [0.82073927] [0.5944743]
20 0.017251031 [0.84745306] [0.34677517]
40 0.0065178988 [0.90623295] [0.21315448]
60 0.002462646 [0.9423636] [0.13102098]
80 0.0009304525 [0.96457225] [0.08053549]
100 0.00035154718 [0.97822344] [0.04950326]
120 0.0001328245 [0.98661447] [0.03042847]
140 5.0184175e-05 [0.99177223] [0.01870367]
160 1.8961251e-05 [0.9949426] [0.01149665]
180 7.163704e-06 [0.9968913] [0.00706669]
200 2.706856e-06 [0.9980892] [0.00434373]
220 1.0226748e-06 [0.9988255] [0.00266992]
240 3.8635457e-07 [0.99927807] [0.00164117]
260 1.4602297e-07 [0.99955624] [0.00100878]
280 5.515631e-08 [0.9997272] [0.00062004]
300 2.0840792e-08 [0.99983233] [0.00038118]
320 7.874145e-09 [0.99989694] [0.00023426]
340 2.9705471e-09 [0.9999367] [0.00014401]
360 1.126447e-09 [0.9999611] [8.851909e-05]
380 4.28012e-10 [0.99997604] [5.442522e-05]
400 1.6060635e-10 [0.9999853] [3.3420547e-05]
420 6.067561e-11 [0.999991] [2.0593629e-05]
440 2.3457384e-11 [0.99999446] [1.2646341e-05]
460 8.73493

In [36]:
# %load simple_linear_regression_gd_tf_placeholder_test.py
import tensorflow as tf

## 데이터 수집

x_data = [1, 2, 3]
y_data = [1, 2, 3]

## 예측 모델 정의

X = tf.placeholder("float")
y = tf.placeholder("float")
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
pred = W * X + b

## 비용 함수, 최적화 함수 정의

cost = tf.reduce_mean(tf.square(pred - y))
a = tf.Variable(0.1) # learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)
# 변수 초기화
init = tf.global_variables_initializer()

## 훈련
sess = tf.Session()
sess.run(init)

for step in range(2001):
    sess.run(train, feed_dict = {X: x_data, y: y_data}) 
    if step % 20 == 0:
        cost_, W_, b_ = sess.run([cost, W, b], feed_dict = {X: x_data, y: y_data}) 
        print(step, cost_, W_, b_)

print(sess.run(pred, feed_dict = {X: [2]}))


0 0.0668651 [1.3160592] [-0.6157036]
20 0.021674579 [1.1709902] [-0.38870093]
40 0.008189242 [1.1051036] [-0.23892525]
60 0.0030941088 [1.0646046] [-0.14686161]
80 0.0011690324 [1.0397109] [-0.09027231]
100 0.00044169548 [1.0244094] [-0.05548818]
120 0.00016688376 [1.0150038] [-0.03410733]
140 6.305319e-05 [1.0092225] [-0.02096491]
160 2.3823499e-05 [1.0056689] [-0.01288664]
180 9.000775e-06 [1.0034845] [-0.00792111]
200 3.4006646e-06 [1.0021418] [-0.00486894]
220 1.2848732e-06 [1.0013165] [-0.0029928]
240 4.853817e-07 [1.0008092] [-0.00183952]
260 1.833924e-07 [1.0004973] [-0.00113062]
280 6.927438e-08 [1.0003058] [-0.0006949]
300 2.6176304e-08 [1.000188] [-0.00042714]
320 9.892749e-09 [1.0001155] [-0.00026251]
340 3.728541e-09 [1.0000709] [-0.00016136]
360 1.4111947e-09 [1.0000436] [-9.9163764e-05]
380 5.356213e-10 [1.0000268] [-6.0897582e-05]
400 2.0094149e-10 [1.0000165] [-3.7437192e-05]
420 7.71602e-11 [1.0000101] [-2.2996976e-05]
440 2.8716585e-11 [1.0000062] [-1.41158835e-05]
46

# 텐서보드 (TensorBoard)

알고리듬이 어떻게 돌아가고 있는지 알려주기 위해 많은 정보를 모니터링하고 디스플레이해준다.

In [ ]:
# %load summary_test.py
import tensorflow as tf

tf.reset_default_graph()

## 데이터 수집

x_data = [1, 2, 3]
y_data = [1, 2, 3]

## 예측 모델 정의

W = tf.get_variable("W", initializer=tf.random_uniform([1], -1.0, 1.0))
b = tf.get_variable("b", initializer=tf.random_uniform([1], -1.0, 1.0))
y = W * x_data + b

## 비용 함수, 최적화 함수 정의

cost = tf.reduce_mean(tf.square(y - y_data))
alpha = tf.get_variable("alpha", initializer=0.01) # learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(cost)

## 초기화, summary and graph log

init = tf.global_variables_initializer()

tf.summary.scalar("cost", cost)         # 텐서
tf.summary.scalar("W", W[0])
tf.summary.scalar("b", b[0])

summary_op = tf.summary.merge_all()     # 모든 텐서 summary를 한꺼번에 모아서 변수에 저장
saver = tf.train.Saver()                # 모델 저장용

sess = tf.Session()     # 텐서플로 세선 
summary_writer = tf.summary.FileWriter("summary_logs/", sess.graph) # 텐서플로의 그래프 객체를 가져와서 저장함
sess.run(init)

## 훈련

for step in range(500):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(cost), sess.run(W), sess.run(b))
    summary_str = sess.run(summary_op)
    summary_writer.add_summary(summary_str, step)       # 저장할 때는 add_summary 함수로 정보를 저장.
    saver.save(sess, "summary_logs/model-checkpoint", global_step=step)    # 세이버에 모델 저장. 1 스텝마다 모델 저장.
    # summary_logs 폴더에 들어가보면 모델이 저장된 것을 확인할 수 있다.


!c:\anaconda3\scripts\tensorboard --logdir summary_logs

http://localhost:6006 방문